<a href="https://colab.research.google.com/github/crychenet/analysis_of_financion_report/blob/master/%D0%9A%D0%BE%D0%BD%D1%82%D1%80%D0%BE%D0%BB%D1%8C_%D0%BF%D1%80%D0%BE%D0%B4%D0%B0%D0%B6_WB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import sys
# pip install xlsxwriter

In [ ]:
initial_df = pd.read_excel('/content/initial_df.xlsx')
last_week_1 = pd.read_excel('/content/47923665.xlsx')
last_week_2 = pd.read_excel('/content/47923666.xlsx')

initial_df = pd.concat([initial_df, last_week_1, last_week_2])

ouer_sales_report = pd.read_excel('/content/2023.10.31 Отчет по скидкам_021123.xlsx', skiprows=1)
warehouse_df = pd.read_excel("/content/Склад 20.xlsx")

price_df = pd.read_excel('/content/actual_price.xlsx')
wb_sales_report = pd.read_excel('/content/Отчет по скидкам (4).xlsx')

In [ ]:
# duplicates = price_df[price_df[['Номенклатура.Артикул']].duplicated(subset='Номенклатура.Артикул', keep=False)][['Номенклатура.Артикул', 'Оптовая РУБ']]
# # duplicates.reset_index(inplace=True, drop=True)
# # for row in duplicates['Номенклатура.Артикул'].unique():
# #     print(duplicates[duplicates['Номенклатура.Артикул'] == row])
# intersection = list(set(list(merge_df['Артикул поставщика'])) & set(list(duplicates['Номенклатура.Артикул'])))

# duplicates.reset_index(inplace=True, drop=True)
# for row in intersection:
#     df = duplicates[duplicates['Номенклатура.Артикул'] == row]['Оптовая РУБ']
#     test = df.iloc[0]
#     are_all_elements_equal = df.eq(test).all()
#     if not are_all_elements_equal:
#         print(df)

In [ ]:
initial_df['Сумма фактических продаж'] = initial_df['Цена розничная с учетом согласованной скидки'] * initial_df['Кол-во']

sales_filter = ['Продажа', 'Сторно возвратов', 'Компенсация подмененного товара', 'Частичная компенсация брака',
                'Корректная продажа']
return_filter = ['Возврат', 'Корректный возврат', 'Сторно продаж']

sales_df = initial_df[initial_df['Обоснование для оплаты'].isin(sales_filter)][['Артикул поставщика', 'Сумма фактических продаж']]
return_df = initial_df[initial_df['Обоснование для оплаты'].isin(return_filter)][['Артикул поставщика', 'Сумма фактических продаж']]

sales_df = sales_df.groupby('Артикул поставщика').sum()
sales_df['Сумма продаж'] = sales_df['Сумма фактических продаж']
sales_df.drop(columns=['Сумма фактических продаж'], inplace=True)
return_df = return_df.groupby('Артикул поставщика').sum()
return_df['Сумма возвратов'] = return_df['Сумма фактических продаж']
return_df.drop(columns=['Сумма фактических продаж'], inplace=True)


merge_df = pd.merge(sales_df, return_df, how='outer', right_index=True, left_index=True)
merge_df.fillna(0, inplace=True)
merge_df['Итог выручки'] = merge_df['Сумма продаж'] - merge_df['Сумма возвратов']

merge_df = pd.merge(
    merge_df, initial_df[initial_df['Обоснование для оплаты'].isin(['Продажа', 'Корректная продажа', 'Сторно возвратов'])].groupby('Артикул поставщика').count()['Кол-во'],
    left_index=True, right_index=True, how='left'
    )
merge_df.rename(columns={'Кол-во': 'Кол-во продаж'}, inplace=True)
merge_df = pd.merge(
    merge_df, initial_df[initial_df['Обоснование для оплаты'].isin(['Возврат', 'Корректный возврат', 'Сторно продаж'])].groupby('Артикул поставщика').count()['Кол-во'],
    left_index=True, right_index=True, how='left'
    )
merge_df.rename(columns={'Кол-во': 'Кол-во возвратов'}, inplace=True)
merge_df.fillna(0, inplace=True)
merge_df['Итог продаж'] = merge_df['Кол-во продаж'] - merge_df['Кол-во возвратов']

merge_df.reset_index(inplace=True)

In [ ]:
wb_sales_report['Актуальная цена'] = wb_sales_report['Текущая розн. цена (до скидки)'] * (1 - wb_sales_report['Текущая скидка на сайте, %'] / 100)
merge_df = pd.merge(merge_df, wb_sales_report[['Артикул продавца', 'Актуальная цена', 'Артикул WB', 'Остаток товара (шт.)', 'Количество дней на сайте', 'Текущая скидка на сайте, %', 'Текущая розн. цена (до скидки)']], right_on='Артикул продавца',left_on='Артикул поставщика' , how='left')

# 16 недель по 7 дней
merge_df['Итог дней в наличии'] = np.where(
    merge_df['Количество дней на сайте'] >= 7 * 16,
    7 * 16,
    merge_df['Количество дней на сайте']
)

merge_df = pd.merge(merge_df, price_df.groupby('Номенклатура.Артикул').max().reset_index()[['Номенклатура.Артикул', 'Оптовая РУБ']],
                    how='left', left_on='Артикул поставщика', right_on='Номенклатура.Артикул')

merge_df = pd.merge(merge_df, ouer_sales_report[['КОЛЛЕКЦИЯ!!!', 'Артикул 1С']], left_on='Артикул поставщика', right_on='Артикул 1С', how='left')

merge_df['РРЦ'] = merge_df['Оптовая РУБ'] * 2

merge_df.rename(columns={'КОЛЛЕКЦИЯ!!!': 'Коллекция', 'Остаток товара (шт.)': 'Остатки WB, шт', 'Текущая розн. цена (до скидки)': 'ПЦ'}, inplace=True)
merge_df.drop(columns=['Оптовая РУБ', 'Номенклатура.Артикул', 'Артикул 1С', 'Артикул продавца'], inplace=True)

merge_df['РРЦ'] = merge_df['РРЦ'].fillna(0)

merge_df['РРЦ'] = np.where(
    merge_df['РРЦ'] == 0,
    merge_df['ПЦ'] / 1.5,
    merge_df['РРЦ']
)

merge_df.fillna(0, inplace=True)
merge_df['Совокупная скидка от РРЦ'] =np.where(
    merge_df['Итог продаж'] * merge_df['РРЦ'] != 0,
    1 - (merge_df['Итог выручки'] / (merge_df['Итог продаж'] * merge_df['РРЦ'])),
    0
)

merge_df['Актуальная скидка от РРЦ'] = np.where(
    merge_df['РРЦ'] != 0,
    1 - (merge_df['Актуальная цена'] / merge_df['РРЦ']),
    0
)

<ipython-input-223-3bf05c1ec260>:11: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  merge_df = pd.merge(merge_df, price_df.groupby('Номенклатура.Артикул').max().reset_index()[['Номенклатура.Артикул', 'Оптовая РУБ']],


In [ ]:
def sort_warehouse(pandas_df):
    def calculate_sum(column):
        # Доделать исключения
        try:
            split_column = column.split(' ')
            sum_size_boxe = split_column[1]
            sum_size_boxe = re.sub(r'[(, )]', '', sum_size_boxe)
            split_column[0] = split_column[0].replace(';', ',')
            return split_column[0] + ' ' + str(sum([int(element) for element in list(sum_size_boxe)])) + ' ' + split_column[1]
        except:
            pass # Обратить внимание
    # ТИПА 23/24;25/26;27/28;29/30 (2222) не должно быть!
    pandas_df['Характеристика номенклатуры.Размеры'] = np.where(
        pandas_df['Характеристика номенклатуры.Размеры'].str.contains(';'),
        pandas_df['Характеристика номенклатуры.Размеры'].str.replace(';', ',').apply(calculate_sum),
        pandas_df['Характеристика номенклатуры.Размеры']
    )
    pandas_df['Характеристика номенклатуры.Размеры'] = pandas_df['Характеристика номенклатуры.Размеры'].astype(str)
    # Удалить строку Итог
    article_information = pandas_df['Характеристика номенклатуры.Размеры'].str.split(' ', expand=True)
    article_information.rename(columns={0: 'Размер', 1: 'Кол-во пар', 2: 'Список кол-ва размеров'}, inplace=True)
    # article_information.to_excel('article_information.xlsx')
    sorted_df = pd.DataFrame({
        'Артикул': pandas_df['Артикул'],
        'Размер': article_information['Размер'],
        'Кол-во пар': article_information['Кол-во пар'],
        'Список кол-ва размеров': article_information['Список кол-ва размеров'],
        'Свободно': pandas_df['Свободно'],
        'Способ доставки': np.where(
            (article_information['Список кол-ва размеров'].str.len() == 4) | (article_information['Список кол-ва размеров'].str.len() == 3),
            'Монокороб',
            np.where(
                article_information['Кол-во пар'].isnull(),
                'Штучно',
                'Ростовой короб'
            ))
    })
    return sorted_df


sorted_warehouse_df = sort_warehouse(warehouse_df)

In [ ]:
merge_df = pd.merge(merge_df, sorted_warehouse_df[['Артикул', 'Свободно']].groupby('Артикул').sum().reset_index(), left_on='Артикул поставщика',
                    right_on='Артикул', how='left')
merge_df.drop(columns=['Артикул'], inplace=True)
merge_df.rename(columns={'Свободно': 'Остатки ЦС, шт'}, inplace=True)
merge_df.fillna(0, inplace=True)

# merged_information_from_mpstats = pd.merge(
#     mpstats_first_period.groupby('Артикул').agg({'Дней в наличии': 'max'})[['Дней в наличии']],
#     mpstats_second_period.groupby('Артикул').agg({'Дней в наличии': 'max'})[['Дней в наличии']],
#     left_index=True, right_index=True, how='outer')
# merged_information_from_mpstats['Итог дней в наличии'] = merged_information_from_mpstats['Дней в наличии_x'] + merged_information_from_mpstats['Дней в наличии_y']

# merge_df = pd.merge(merge_df, merged_information_from_mpstats.reset_index()[['Артикул', 'Итог дней в наличии']],
#                     left_on='Артикул поставщика', right_on='Артикул', how='left')

# 8 дней на 15 недель
merge_df['Скорость продаж'] = np.where(
    merge_df['Итог дней в наличии'] != 0,
    merge_df['Итог продаж'] / merge_df['Итог дней в наличии'],
    0
)

merge_df['Итог остатков, шт (нужно распродать)'] = merge_df['Остатки ЦС, шт'] * 0.5 + merge_df['Остатки WB, шт']
merge_df['Оборачиваемость'] = np.where(
    merge_df['Скорость продаж'] > 0,
    merge_df['Итог остатков, шт (нужно распродать)'] / merge_df['Скорость продаж'],
    'INF'
)

In [ ]:
vectorized_get_hyperlink_WB = np.vectorize(lambda column: f"https://www.wildberries.ru/catalog/{int(column)}/detail.aspx?targetUrl=SP")
merge_df['Ссылка WB'] = vectorized_get_hyperlink_WB(merge_df['Артикул WB'])

merge_df = pd.merge(merge_df, initial_df.drop_duplicates('Артикул поставщика')[['Предмет', 'Бренд', 'Артикул поставщика']], on='Артикул поставщика', how='left')

vectorized_get_hyperlink_mpstat = np.vectorize(lambda column: f"https://mpstats.io/wb/item/{int(column)}")
merge_df['Ссылка MPSTATS'] = vectorized_get_hyperlink_mpstat(merge_df['Артикул WB'])

merge_df.fillna(0, inplace=True)
merge_df['Оборачиваемость'] = merge_df['Оборачиваемость'].astype(str).replace('INF', '9999')
merge_df['Итог дней в наличии'] = merge_df['Итог дней в наличии'].astype(float)
merge_df['Оборачиваемость'] = merge_df['Оборачиваемость'].astype(float)

In [ ]:
merge_df['sellThru'] = np.where(
    merge_df['Итог продаж'] * 2 / (merge_df['Итог остатков, шт (нужно распродать)'] + merge_df['Итог продаж']) > 1,
    1,
    np.where(
        merge_df['Итог продаж'] * 2 / (merge_df['Итог остатков, шт (нужно распродать)'] + merge_df['Итог продаж']) <= 0,
        0,
        merge_df['Итог продаж'] * 2 / (merge_df['Итог остатков, шт (нужно распродать)'] + merge_df['Итог продаж'])
    )
)


In [ ]:
merge_df['Статус'] = np.where(
    0.9 < merge_df["sellThru"],
    np.where(
        merge_df["Совокупная скидка от РРЦ"] < 0.05,
        "A1",
        np.where(
            (0.05 <= merge_df["Совокупная скидка от РРЦ"]) & (merge_df["Совокупная скидка от РРЦ"] < 0.2),
            "A2",
            np.where(
                (0.2 <= merge_df["Совокупная скидка от РРЦ"]) & (merge_df["Совокупная скидка от РРЦ"] < 0.4),
                "A3",
                np.where(
                    0.4 <= merge_df["Совокупная скидка от РРЦ"],
                    "A4",
                    "Error"
                )
            )
        )
    ),
    np.where(
        (0.8 < merge_df["sellThru"]) & (merge_df["sellThru"] <= 0.9),
        np.where(
            merge_df["Совокупная скидка от РРЦ"] < 0.05,
            "B1",
            np.where(
                (0.05 <= merge_df["Совокупная скидка от РРЦ"]) & (merge_df["Совокупная скидка от РРЦ"] < 0.2),
                "B2",
                np.where(
                    (0.2 <= merge_df["Совокупная скидка от РРЦ"]) & (merge_df["Совокупная скидка от РРЦ"] < 0.4),
                    "B3",
                    np.where(
                        (0.4 <= merge_df["Совокупная скидка от РРЦ"]),
                        "B4",
                        "Error"
                    )
                )
            )
        ),
        np.where(
            (0.6 < merge_df["sellThru"]) & (merge_df["sellThru"] <= 0.8),
            np.where(
                merge_df["Совокупная скидка от РРЦ"] < 0.05,
                "C1",
                np.where(
                    (0.05 <= merge_df["Совокупная скидка от РРЦ"]) & (merge_df["Совокупная скидка от РРЦ"] < 0.2),
                    "C2",
                    np.where(
                        (0.2 <= merge_df["Совокупная скидка от РРЦ"]) & (merge_df["Совокупная скидка от РРЦ"] < 0.4),
                        "C3",
                        np.where(
                            0.4 <= merge_df["Совокупная скидка от РРЦ"],
                            "C4",
                            "Error"
                        )
                    )
                )
            ),
            np.where(
                merge_df["sellThru"] <= 0.6,
                np.where(
                    merge_df["Совокупная скидка от РРЦ"] < 0.05,
                    "D1",
                    np.where(
                        (0.05 <= merge_df["Совокупная скидка от РРЦ"]) & (merge_df["Совокупная скидка от РРЦ"] < 0.2),
                        "D2",
                        np.where(
                            (0.2 <= merge_df["Совокупная скидка от РРЦ"]) & (merge_df["Совокупная скидка от РРЦ"] < 0.4),
                            "D3",
                            np.where(
                                0.4 <= merge_df["Совокупная скидка от РРЦ"],
                                "D4",
                                "Error"
                            )
                        )
                    )
                ),
                "Error"
            )
        )
    )
)

In [ ]:
merge_df['Новая цена'] = 1

merge_df['Новая цена'] = np.where(
    merge_df['Статус'] == 'A1',
    merge_df['Актуальная цена'] * 1.05,
      np.where(
        merge_df['Статус'] == 'A2',
        merge_df['Актуальная цена'] * 1.05,
        np.where(
            merge_df['Статус'] == 'A3',
            merge_df['Актуальная цена'] * 1.1,
            np.where(
                merge_df['Статус'] == 'A4',
                merge_df['Актуальная цена'] * 1.1,
                merge_df['Новая цена']
            )
        )
    )
)

merge_df['Новая цена'] = np.where(
    merge_df['Статус'] == 'B1',
    merge_df['Актуальная цена'],
      np.where(
        merge_df['Статус'] == 'B2',
        merge_df['Актуальная цена'],
        np.where(
            merge_df['Статус'] == 'B3',
            merge_df['Актуальная цена'],
            np.where(
                merge_df['Статус'] == 'B4',
                merge_df['Актуальная цена'] * 1.05,
                merge_df['Новая цена']
            )
        )
    )
)

merge_df['Новая цена'] = np.where(
    merge_df['Статус'] == 'C1',
    merge_df['Актуальная цена'] * 0.85,
      np.where(
        merge_df['Статус'] == 'C2',
        merge_df['РРЦ'] * 0.7,
        np.where(
            merge_df['Статус'] == 'C3',
            merge_df['РРЦ'] * 0.6,
            np.where(
                merge_df['Статус'] == 'C4',
                merge_df['РРЦ'] * 0.5,
                merge_df['Новая цена']
            )
        )
    )
)

merge_df['Новая цена'] = np.where(
    merge_df['Статус'] == 'D1',
    merge_df['РРЦ'] * 0.8,
      np.where(
        merge_df['Статус'] == 'D2',
        merge_df['РРЦ'] * 0.7,
        np.where(
            merge_df['Статус'] == 'D3',
            merge_df['РРЦ'] * 0.6,
            np.where(
                merge_df['Статус'] == 'D4',
                merge_df['РРЦ'] * 0.5,
                merge_df['Новая цена']
            )
        )
    )
)

merge_df['Новая цена'] = np.where(
    merge_df['Итог дней в наличии'] > 14,
    merge_df['Новая цена'],
    merge_df['Актуальная цена']
)

In [ ]:
merge_df['Новинка'] = np.where(
    merge_df['Итог дней в наличии'] > 14,
    'Нет',
    'Да'
)

merge_df['Новая скидка от ПЦ'] = np.where(
    merge_df['ПЦ'] == 0,
    0,
    1 - (merge_df['Новая цена'] / merge_df['ПЦ'])
)


In [ ]:
merge_df['Ценообразование'] = 1

merge_df['Ценообразование'] = np.where(
    merge_df['Статус'] == 'A1',
    'Поднимаем цены на 5% к текущей цене',
    np.where(
        merge_df['Статус'] == 'A2',
        'Поднимаем цены на 5% к текущей цене',
        np.where(
            merge_df['Статус'] == 'A3',
            'Поднимаем цены на 10% к текущей цене',
            np.where(
                merge_df['Статус'] == 'A4',
                'Поднимаем цены на 10% к текущей цене',
                merge_df['Новая цена']
            )
        )
    )
)

merge_df['Ценообразование'] = np.where(
    merge_df['Статус'] == 'B1',
    'Цену не меняем',
    np.where(
        merge_df['Статус'] == 'B2',
        'Цену не меняем',
        np.where(
            merge_df['Статус'] == 'B3',
            'Цену не меняем',
            np.where(
                merge_df['Статус'] == 'B4',
                'Поднимаем цены на 5% к текущей цене',
                merge_df['Ценообразование']
            )
        )
    )
)

merge_df['Ценообразование'] = np.where(
    merge_df['Статус'] == 'C1',
    'Снижаем цену на 15%',
    np.where(
        merge_df['Статус'] == 'C2',
        'Скидка 30%',
        np.where(
            merge_df['Статус'] == 'C3',
            'Скидка 40%',
            np.where(
                merge_df['Статус'] == 'C4',
                'Скидка 50%',
                merge_df['Ценообразование']
            )
        )
    )
)

merge_df['Ценообразование'] = np.where(
    merge_df['Статус'] == 'D1',
    'Скидка 20%',
    np.where(
        merge_df['Статус'] == 'D2',
        'Скидка 30%',
        np.where(
            merge_df['Статус'] == 'D3',
            'Скидка 40%',
            np.where(
                merge_df['Статус'] == 'D4',
                'Скидка 50%',
                merge_df['Ценообразование']
            )
        )
    )
)

In [ ]:
# "Текущая скидка на сайте, %" должна быть предпоследней колонкой и поменять местами "Итог остатков, шт (нужно распродать)" и "Остатки ЦС, шт"

merge_df = pd.merge(merge_df, ouer_sales_report[['Артикул WB', 'КОЛЛЕКЦИЯ!!!']], on='Артикул WB', how='left')
merge_df.drop(columns=['Коллекция'], inplace=True)
merge_df.rename(columns={'КОЛЛЕКЦИЯ!!!': 'Коллекция'}, inplace=True)
merge_df = merge_df.iloc[:, [0, 8, 22, 23, 31, 21, 24, 3, 6, 9, 17, 10, 13, 12, 14, 7, 15, 16, 19, 18, 20, 25, 28, 26, 30, 27, 11, 29]]


with pd.ExcelWriter('Контроль продаж WB 22.11 Итог.xlsx', engine='xlsxwriter') as writer:
    merge_df.to_excel(writer, sheet_name='Расчет', index=False)